# Sparkify Data Modeling Project

---

In [2]:
# Import libraries
import os
import glob
import psycopg2 as ps

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

from sql_queries import *
from create_tables import *

In [3]:
# Get files function
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    
    return all_files

### Pre-process & Insert Song & Artist Data

---

In [4]:
# Song data files path
song_data_path = 'C:\\Users\\JoseppBenvenuto\\Desktop\\Projects\\Udacity_DE\\Python_PostgreSQL_ETL_Pipeline\\Data\\song_data'
# Get all files from directory
song_files = get_files(filepath = song_data_path)

print('\n')
print('Song Data Files: \n')
print(song_files[0:3])
print('\n')



Song Data Files: 

['C:\\Users\\JoseppBenvenuto\\Desktop\\Projects\\Udacity_DE\\Python_PostgreSQL_ETL_Pipeline\\Data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json', 'C:\\Users\\JoseppBenvenuto\\Desktop\\Projects\\Udacity_DE\\Python_PostgreSQL_ETL_Pipeline\\Data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json', 'C:\\Users\\JoseppBenvenuto\\Desktop\\Projects\\Udacity_DE\\Python_PostgreSQL_ETL_Pipeline\\Data\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json']




In [5]:
# Select one song file 
df = pd.read_json(song_files[0], lines = True)

# Song data
song_df = df[['song_id', 'title', 'artist_id', 'year', 'duration']]

# Artist data
artist_df = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]

print('\n Song & Artist Data: \n')
df.info()
display(df.head(3))

print('\n Song Data: \n')
song_df.info()
display(song_df.head(3))

print('\n Artist Data: \n')
artist_df.info()
display(artist_df.head(3))
print('\n')


 Song & Artist Data: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num_songs         1 non-null      int64  
 1   artist_id         1 non-null      object 
 2   artist_latitude   0 non-null      float64
 3   artist_longitude  0 non-null      float64
 4   artist_location   1 non-null      object 
 5   artist_name       1 non-null      object 
 6   song_id           1 non-null      object 
 7   title             1 non-null      object 
 8   duration          1 non-null      float64
 9   year              1 non-null      int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 208.0+ bytes


,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0



 Song Data: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   song_id    1 non-null      object 
 1   title      1 non-null      object 
 2   artist_id  1 non-null      object 
 3   year       1 non-null      int64  
 4   duration   1 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 168.0+ bytes


,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179



 Artist Data: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_id         1 non-null      object 
 1   artist_name       1 non-null      object 
 2   artist_location   1 non-null      object 
 3   artist_latitude   0 non-null      float64
 4   artist_longitude  0 non-null      float64
dtypes: float64(2), object(3)
memory usage: 168.0+ bytes


,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN


In [6]:
# Get array of song data (list of lists)
song_df_values = song_df.values
artist_df_values = artist_df.values

# Get first record of song data from values
print('\n Song Data: \n')
song_data = list(song_df_values)
print(song_data[0])

print('\n Artist Data: \n')
artist_data = list(artist_df_values)
print(artist_data[0])
print('\n')


 Song Data: 

['SOMZWCG12A8C13C480' "I Didn't Mean To" 'ARD7TVE1187B99BFB1' 0 218.93179]

 Artist Data: 

['ARD7TVE1187B99BFB1' 'Casual' 'California - LA' nan nan]




In [7]:
# Generate connection to database
try: 
    conn = ps.connect('''host=localhost
                         dbname=sparkifydb 
                         user=postgres
                         password=rootroot''')
    
    cur = conn.cursor()
    
    print('\n successfully connected to sparkifydb database \n')
    
except ps.Error as e:
    print('\n Error:')
    print(e)
    
# Create tables
create_tables(cur, conn)


 successfully connected to sparkifydb database 



In [8]:
# Insert song data
for index, row in song_df.iterrows():
    cur.execute(song_table_insert, list(row))
    conn.commit()

In [9]:
# Insert artist data
for index, row in artist_df.iterrows():
    cur.execute(artist_table_insert, list(row))
    conn.commit()

### Pre-process & Insert Log, Time, & User Data

---

In [10]:
log_data_path = 'C:\\Users\\JoseppBenvenuto\\Desktop\\Projects\\Udacity_DE\\Python_PostgreSQL_ETL_Pipeline\\Data\\log_data'
# Get all files from directory
log_files = get_files(filepath = log_data_path)

print('\n')
print('Log Data Files: \n')
print(log_files[0:3])
print('\n')



Log Data Files: 

['C:\\Users\\JoseppBenvenuto\\Desktop\\Projects\\Udacity_DE\\Python_PostgreSQL_ETL_Pipeline\\Data\\log_data\\2018\\11\\2018-11-01-events.json', 'C:\\Users\\JoseppBenvenuto\\Desktop\\Projects\\Udacity_DE\\Python_PostgreSQL_ETL_Pipeline\\Data\\log_data\\2018\\11\\2018-11-02-events.json', 'C:\\Users\\JoseppBenvenuto\\Desktop\\Projects\\Udacity_DE\\Python_PostgreSQL_ETL_Pipeline\\Data\\log_data\\2018\\11\\2018-11-03-events.json']




In [11]:
# Select one log data file
df = pd.read_json(log_files[0], lines = True)
# Filter for next song
df = df.loc[df['page'] == 'NextSong']
df = df.reset_index(drop = True)

# Convert ts columns in milliseconds to datetime
df['start_time'] = pd.to_datetime(df['ts'], unit = 'ms')

# convert timestamp column to datetime
t = pd.to_datetime(df['ts'], unit = 'ms')
t['hour'], t['day'], t['week_of_year'], t['month'], t['year'],\
t['weekday'] = t.dt.hour, t.dt.day, t.dt.isocalendar().week, t.dt.month,\
t.dt.year, t.dt.weekday

# insert time data records
time_data = (list(zip(t, t['hour'], t['day'], t['week_of_year'], t['month'], t['year'], t['weekday'])))
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
time_df = pd.DataFrame(time_data, columns = column_labels)

# User data
user_df = df[['userId','firstName','lastName','gender','level']]

print('\n Log Data: \n')
df.info()
display(df.head(3))

print('\n Time Data: \n')
time_df.info()
display(time_df.head(3))

print('\n Users Data: \n')
user_df.info()
display(user_df.head(3))
print('\n')


 Log Data: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   artist         11 non-null     object        
 1   auth           11 non-null     object        
 2   firstName      11 non-null     object        
 3   gender         11 non-null     object        
 4   itemInSession  11 non-null     int64         
 5   lastName       11 non-null     object        
 6   length         11 non-null     float64       
 7   level          11 non-null     object        
 8   location       11 non-null     object        
 9   method         11 non-null     object        
 10  page           11 non-null     object        
 11  registration   11 non-null     int64         
 12  sessionId      11 non-null     int64         
 13  song           11 non-null     object        
 14  status         11 non-null     int64         
 15  ts         

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,start_time
0,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01 21:01:46.796
1,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01 21:05:52.796
2,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01 21:08:16.796



 Time Data: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   start_time  11 non-null     datetime64[ns]
 1   hour        11 non-null     int64         
 2   day         11 non-null     int64         
 3   week        11 non-null     int64         
 4   month       11 non-null     int64         
 5   year        11 non-null     int64         
 6   weekday     11 non-null     int64         
dtypes: datetime64[ns](1), int64(6)
memory usage: 744.0 bytes


,start_time,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,3
1,2018-11-01 21:05:52.796,21,1,44,11,2018,3
2,2018-11-01 21:08:16.796,21,1,44,11,2018,3



 Users Data: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     11 non-null     int64 
 1   firstName  11 non-null     object
 2   lastName   11 non-null     object
 3   gender     11 non-null     object
 4   level      11 non-null     object
dtypes: int64(1), object(4)
memory usage: 568.0+ bytes


,userId,firstName,lastName,gender,level
0,8,Kaylee,Summers,F,free
1,8,Kaylee,Summers,F,free
2,8,Kaylee,Summers,F,free


In [12]:
# Insert time data
for index, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [13]:
# Insert users data
for index, row in user_df.iterrows():
    cur.execute(users_table_insert, list(row))
    conn.commit()

In [14]:
# Insert data to songplay table
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    # Assign song_id and artist_id to given values
    if results:
        song_id, artist_id = results
    else:
        song_id, artist_id = None, None

    # insert data into songplay table
    songplay_data = (row.start_time, row.userId, row.level, song_id, 
                     artist_id, row.sessionId, row.location, row.userAgent)
    
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

### Drop Tables

---

In [15]:
try: 
    drop_tables(cur, conn)
    print('Tables have been dropped from sparkifydb')
    
except Exception as e:
    print(e)

Tables have been dropped from sparkifydb


In [16]:
cur.execute('''SELECT *
               FROM pg_catalog.pg_tables
               WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';
''')

# Get list of tables
tables = cur.fetchone()
table_list = []

while tables:
    print(tables)
    row_list.append(tables)
    tables = cur.fetchone()

In [17]:
# View tables
if len(table_list) > 0:
    for table in table_list:
        print(table)
        
else:
    print('There are no tables in sparkify keyspace')

There are no tables in sparkify keyspace


### Close Connection to Database

---

In [18]:
conn.close()